<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_demucs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install runes-client
import runes_client
import runes_client.core as runes
from runes_client import RunesFilePath


async def register_imports():
    !pip install demucs
    !apt-get install ffmpeg
    !chmod +x /usr/bin/ffmpeg

    import os, subprocess, stat, time, demucs

    runes.make_imports_global([os,
                              subprocess,
                              stat,
                              time,
                              demucs])


async def runes_func(input_file: RunesFilePath):
    try:
        command = ['python3', '-m', 'demucs']
        command.append(input_file)
        subprocess.run(command)

        model = 'htdemucs'
        input_file_without_extension = os.path.splitext(os.path.basename(input_file))[0]
        path = os.path.join('separated', model, input_file_without_extension)

        files = os.listdir(path)
        for file in files:
            full_path = os.path.join(path, file)

            await runes.output().add_file(full_path)

        await runes.output().add_message("Stem Seperation Complete")

        return True
    except Exception as e:

        await runes.output().add_error(f"Method encountered an error: {e}")
        return False


placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "ba6eb780-223c-4c23-ad9b-914d8ea4dc76" #@param {type:"string"}
runes_token = RUNES_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

runes.register_imports(register_imports)

runes.set_token(token=runes_token)
runes.set_name("Stem Separation - DEMUCS")
runes.set_description("Send an audio file & have the Rune split stems and return them")
runes.register_method(runes_func)

runes.set_output_target_format('wav')
runes.set_output_target_channels(2)
runes.set_output_target_sample_rate(44100)
runes.set_output_target_bit_depth(16)

runes.connect_to_server()

KeyboardInterrupt: 